# Apply Association rules to financial markets data

In [1]:
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

Import market data

In [179]:
fx = pd.read_csv('FXdata.csv')
FwdsImplYield = pd.read_csv('FX_Fwds_ImpliedDepo_data.csv')
OIS = pd.read_csv('OIS_data.csv')
LIBOR = pd.read_csv('LIBOR_data.csv')
Equities = pd.read_csv('Equities_data.csv')
Bonds = pd.read_csv('BondYields.csv')
Oil = pd.read_csv('Oil.csv')

Join, clean and transform data

In [180]:
# Drop unused/repetitive columns
print(fx.shape)
ccySet = set(['EUR','GBP','AUD','JPY'])
for c in fx.columns:
    if c.find('1YO=')!=-1 or c.find('1YRR=')!=-1 or c.find('1YBF=')!=-1:
        fx.drop(c, axis=1, inplace=True)
    elif (c.find('1MO=')!=-1 or c.find('1MRR=')!=-1 or c.find('1MBF=')!=-1) and not c[:3] in ccySet:
        fx.drop(c, axis=1, inplace=True)
print(fx.shape)

(5646, 79)
(5646, 25)


In [181]:
#rename columns (Reuters to market convention)
bipSet = set(['EUR=','GBP=','AUD=','NZD=','XAU='])
for c in fx.columns:
    if len(c)==4 and c[3:]=="=":
        if c in bipSet:
            fx.rename(columns={c:c[:3] + "USD"},inplace=True)
        else:
            fx.rename(columns={c:"USD" + c[:3]},inplace=True)
# drop constituents of DXY index
fx.drop('EURUSD', axis=1, inplace=True) #weight 57.6%
fx.drop('USDJPY', axis=1, inplace=True) #13.6
fx.drop('GBPUSD', axis=1, inplace=True) #11.9
fx.drop('USDCAD', axis=1, inplace=True) #9.1
fx.drop('USDSEK', axis=1, inplace=True) #4.2
fx.drop('USDCHF', axis=1, inplace=True) #3.6

fx.drop('USDDKK', axis=1, inplace=True) #pegged to EUR so essentially inverted EURUSD            
fx.columns

Index(['Timestamp', 'AUDUSD', 'NZDUSD', 'USDNOK', 'XAUUSD', '.DXY', 'EUR1MO=',
       'GBP1MO=', 'JPY1MO=', 'AUD1MO=', 'EUR1MRR=', 'GBP1MRR=', 'JPY1MRR=',
       'AUD1MRR=', 'EUR1MBF=', 'GBP1MBF=', 'JPY1MBF=', 'AUD1MBF='],
      dtype='object')

In [182]:
# FwdsImplYield: convert complete term structure into level, slope and curvature
# these 3 factors explain >90% of the term structure due to high correlation across tenors
for c in FwdsImplYield.columns:
    if c.find('1M')!=-1:
        FwdsImplYield[c[:3] + '_slope'] = FwdsImplYield[c.replace('1M','1Y')]-FwdsImplYield[c]
    elif c.find('3M')!=-1:
        ccy = c[:3]
        #LIBOR-OIS
        if ccy=="USD" or ccy=="GBP" or ccy=="CHF":
            FwdsImplYield[ccy + '_LIBOR_OIS'] = LIBOR[ccy + '3MFSR='] - OIS[ccy + '3MOIS=']
        elif ccy=="EUR":
            FwdsImplYield[ccy + '_LIBOR_OIS'] = LIBOR[ccy + '3MFSR='] - OIS['EUREON3M=']
        elif ccy=="JPY":
            FwdsImplYield[ccy + '_LIBOR_OIS'] = LIBOR[ccy + '3MFSR='] - OIS[ccy + '3MOIS=FMD']
        #FXS-OIS
        if ccy=="GBP" or ccy=="CHF" or ccy=="CAD" or ccy=="AUD" or ccy=="NZD":
            FwdsImplYield[ccy + '_FXS_OIS'] = FwdsImplYield[c] - OIS[ccy + '3MOIS=']
        elif ccy=="JPY":
            FwdsImplYield[ccy + '_FXS_OIS'] = FwdsImplYield[c] - OIS[ccy + '3MOIS=FMD']
        elif ccy=="EUR":
            FwdsImplYield[ccy + '_FXS_OIS'] = FwdsImplYield[c] - OIS['EUREON3M=']
        elif ccy=="SEK":
            FwdsImplYield[ccy + '_FXS_OIS'] = FwdsImplYield[c] - OIS['SEKAMTNS3M=']
    elif c.find('6M')!=-1:
        FwdsImplYield[c[:3] + '_curve']= 1/(5+6)*(5*(FwdsImplYield[c.replace('6M','1Y')]-FwdsImplYield[c])-6*(FwdsImplYield[c]-FwdsImplYield[c.replace('6M','1M')])) # curvature
    elif c.find('1Y')!=-1:
        FwdsImplYield.rename(columns={c:c[:3] + "_1Y"},inplace=True)           #level
# drop columns
for c in FwdsImplYield.columns:
    if c.find('_')==-1 and c!='Timestamp':
        FwdsImplYield.drop(c, axis=1, inplace=True)
print(FwdsImplYield.columns)
FwdsImplYield.head()

Index(['Timestamp', 'EUR_1Y', 'GBP_1Y', 'JPY_1Y', 'CHF_1Y', 'CAD_1Y', 'AUD_1Y',
       'NZD_1Y', 'SEK_1Y', 'NOK_1Y', 'DKK_1Y', 'EUR_slope', 'EUR_LIBOR_OIS',
       'EUR_FXS_OIS', 'EUR_curve', 'GBP_slope', 'GBP_LIBOR_OIS', 'GBP_FXS_OIS',
       'GBP_curve', 'JPY_slope', 'JPY_LIBOR_OIS', 'JPY_FXS_OIS', 'JPY_curve',
       'CHF_slope', 'CHF_LIBOR_OIS', 'CHF_FXS_OIS', 'CHF_curve', 'CAD_slope',
       'CAD_FXS_OIS', 'CAD_curve', 'AUD_slope', 'AUD_FXS_OIS', 'AUD_curve',
       'NZD_slope', 'NZD_FXS_OIS', 'NZD_curve', 'SEK_slope', 'SEK_FXS_OIS',
       'SEK_curve', 'NOK_slope', 'NOK_curve', 'DKK_slope', 'DKK_curve'],
      dtype='object')


,Timestamp,EUR_1Y,GBP_1Y,JPY_1Y,CHF_1Y,CAD_1Y,AUD_1Y,NZD_1Y,SEK_1Y,NOK_1Y,...,NZD_slope,NZD_FXS_OIS,NZD_curve,SEK_slope,SEK_FXS_OIS,SEK_curve,NOK_slope,NOK_curve,DKK_slope,DKK_curve
0,08/02/2019,-0.1725,1.1640,-0.2450,-0.5675,NaN,2.3930,2.2765,0.0940,1.5280,...,0.5390,0.2875,-0.110500,0.3395,0.0655,0.000818,0.6175,-0.107818,0.3015,-0.021955
1,07/02/2019,-0.1850,1.1595,-0.2375,-0.5905,2.2030,2.4275,2.2680,0.0875,1.5240,...,0.5240,0.2705,-0.105818,0.3460,0.0410,0.006773,0.6120,-0.107818,0.3085,-0.042773
2,06/02/2019,-0.1685,1.2015,-0.0875,-0.5670,2.2535,2.4515,2.3455,0.1005,1.5370,...,0.5590,NaN,-0.105409,0.3560,0.0945,0.006318,0.6545,-0.104500,0.2720,-0.020864
3,05/02/2019,-0.1510,1.2290,-0.1630,-0.5420,2.2900,2.4865,2.4050,0.1350,1.5745,...,0.6165,0.3250,-0.100773,0.3850,0.1010,0.006500,0.6970,-0.111182,0.3050,-0.085864
4,04/02/2019,-0.1560,1.2395,-0.1990,-0.5475,2.2755,2.5025,2.3790,0.1205,1.5720,...,0.6145,0.3100,-0.124182,0.4015,0.0780,-0.014000,0.7075,-0.146909,0.2825,-0.024091


In [183]:
for c in Bonds.columns:
    if c.find('2Y')!=-1:
        Bonds[c.replace(' 2Y','_slope')] = Bonds[c.replace(' 2Y',' 10Y')] - Bonds[c]
        Bonds.rename(columns={c:c.replace(' 2Y','_2Y')},inplace=True)
        
# drop columns
for c in Bonds.columns:
    if c.find('_')==-1 and c!='Timestamp':
        Bonds.drop(c, axis=1, inplace=True)
print(Bonds.columns)

Index(['Timestamp', 'US_2Y', 'Ger_2Y', 'It_2Y', 'Jp_2Y', 'GB_2Y', 'US_slope',
       'Ger_slope', 'It_slope', 'Jp_slope', 'GB_slope'],
      dtype='object')


In [184]:
data = pd.merge(fx,FwdsImplYield,how='outer',on='Timestamp',validate="one_to_one")
data = pd.merge(data,Equities,how='outer',on='Timestamp',validate="one_to_one")
data = pd.merge(data,Bonds,how='outer',on='Timestamp',validate="one_to_one")
data = pd.merge(data,Oil,how='outer',on='Timestamp',validate="one_to_one")
print(data.shape)
data.head()

(6311, 79)


,Timestamp,AUDUSD,NZDUSD,USDNOK,XAUUSD,.DXY,EUR1MO=,GBP1MO=,JPY1MO=,AUD1MO=,...,Ger_2Y,It_2Y,Jp_2Y,GB_2Y,US_slope,Ger_slope,It_slope,Jp_slope,GB_slope,WTI
0,08/02/2019,0.7085,0.6748,8.6071,1310.9800,96.576,6.015,9.955,6.050,8.550,...,-0.575,0.594,-0.164,0.716,0.1745,0.686,2.347,0.136,0.464,52.45
1,07/02/2019,0.7099,0.6746,8.5946,1310.0300,96.507,5.900,9.775,6.215,8.515,...,-0.581,0.600,-0.153,0.723,0.1750,0.739,2.348,0.145,0.455,52.60
2,06/02/2019,0.7103,0.6773,8.5408,1306.1500,96.390,5.668,10.060,5.992,8.387,...,-0.556,0.477,-0.158,0.733,0.1760,0.720,2.360,0.144,0.482,53.88
3,05/02/2019,0.7234,0.6892,8.4831,1314.9500,96.067,5.560,10.100,5.950,8.012,...,-0.566,0.461,-0.164,0.760,0.1820,0.732,2.334,0.151,0.471,53.69
4,04/02/2019,0.7224,0.6883,8.4584,1311.6899,95.852,5.488,9.660,6.105,7.807,...,-0.572,0.439,-0.169,0.784,0.1920,0.752,2.314,0.156,0.494,54.83


In [185]:
data['Timestamp'] = pd.to_datetime(data['Timestamp'],dayfirst=True) #string to dateTime
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6311 entries, 0 to 6310
Data columns (total 79 columns):
Timestamp              6311 non-null datetime64[ns]
AUDUSD                 5317 non-null float64
NZDUSD                 5315 non-null float64
USDNOK                 5315 non-null float64
XAUUSD                 5305 non-null float64
.DXY                   5259 non-null float64
EUR1MO=                5238 non-null float64
GBP1MO=                5300 non-null float64
JPY1MO=                5515 non-null float64
AUD1MO=                5310 non-null float64
EUR1MRR=               2231 non-null float64
GBP1MRR=               2232 non-null float64
JPY1MRR=               2227 non-null float64
AUD1MRR=               1602 non-null float64
EUR1MBF=               2201 non-null float64
GBP1MBF=               2136 non-null float64
JPY1MBF=               2222 non-null float64
AUD1MBF=               2191 non-null float64
EUR_1Y                 4049 non-null float64
GBP_1Y                 4044 non

In [186]:
data.head()

,Timestamp,AUDUSD,NZDUSD,USDNOK,XAUUSD,.DXY,EUR1MO=,GBP1MO=,JPY1MO=,AUD1MO=,...,Ger_2Y,It_2Y,Jp_2Y,GB_2Y,US_slope,Ger_slope,It_slope,Jp_slope,GB_slope,WTI
0,2019-02-08,0.7085,0.6748,8.6071,1310.9800,96.576,6.015,9.955,6.050,8.550,...,-0.575,0.594,-0.164,0.716,0.1745,0.686,2.347,0.136,0.464,52.45
1,2019-02-07,0.7099,0.6746,8.5946,1310.0300,96.507,5.900,9.775,6.215,8.515,...,-0.581,0.600,-0.153,0.723,0.1750,0.739,2.348,0.145,0.455,52.60
2,2019-02-06,0.7103,0.6773,8.5408,1306.1500,96.390,5.668,10.060,5.992,8.387,...,-0.556,0.477,-0.158,0.733,0.1760,0.720,2.360,0.144,0.482,53.88
3,2019-02-05,0.7234,0.6892,8.4831,1314.9500,96.067,5.560,10.100,5.950,8.012,...,-0.566,0.461,-0.164,0.760,0.1820,0.732,2.334,0.151,0.471,53.69
4,2019-02-04,0.7224,0.6883,8.4584,1311.6899,95.852,5.488,9.660,6.105,7.807,...,-0.572,0.439,-0.169,0.784,0.1920,0.752,2.314,0.156,0.494,54.83


Close to close change: up, unchanged or down

In [187]:
# create new df with XXX_Up and XXX_Down; boolean values
data.fillna(method='backfill')
DailyChg = data - data.shift(periods=1, axis='rows')
DailyChg=DailyChg.drop('Timestamp', axis=1)
DailyChg.drop(DailyChg.index[0], inplace=True)
#DailyChg=DailyChg.dropna(axis=0)
print(DailyChg.shape)
#DailyChg.head()

(6310, 78)


In [188]:
for c in DailyChg.columns:
    DailyChg[c + '_Up'] = DailyChg[c] > 0
    DailyChg[c + '_Down'] = DailyChg[c] < 0
    DailyChg.drop(c, axis=1, inplace=True)
DailyChg.head()

,AUDUSD_Up,AUDUSD_Down,NZDUSD_Up,NZDUSD_Down,USDNOK_Up,USDNOK_Down,XAUUSD_Up,XAUUSD_Down,.DXY_Up,.DXY_Down,...,Ger_slope_Up,Ger_slope_Down,It_slope_Up,It_slope_Down,Jp_slope_Up,Jp_slope_Down,GB_slope_Up,GB_slope_Down,WTI_Up,WTI_Down
1,True,False,False,True,False,True,False,True,False,True,...,True,False,True,False,True,False,False,True,True,False
2,True,False,True,False,False,True,False,True,False,True,...,False,True,True,False,False,True,True,False,True,False
3,True,False,True,False,False,True,True,False,False,True,...,True,False,False,True,True,False,False,True,False,True
4,False,True,False,True,False,True,False,True,False,True,...,True,False,False,True,True,False,True,False,True,False
5,True,False,True,False,False,True,True,False,False,True,...,False,True,False,True,False,True,False,True,True,False


# Most common sets market conditions

Determining the most frequent combination of market close to close moves (itemset)
Support(itemset) = percentage of trading days for which set of 'close to close' changes in market prices (binary up or down) occurred.
Using lambda function to only show combinations of market changes (itemsets of length >=2)

In [189]:
frequent_itemsets = apriori(DailyChg, min_support=0.15, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets = frequent_itemsets[ (frequent_itemsets['length'] == 2)]
frequent_itemsets.sort_values(by=['support','length'],ascending=False,inplace=True)
frequent_itemsets

,support,itemsets,length
202,0.336767,"(AUDUSD_Down, NZDUSD_Down)",2
2304,0.336450,"(VIX_Up, S&P 500 (USD)_Down)",2
2358,0.326624,"(EURO STOXX 50 VOL_Up, EURO STOXX 50 (EUR)_Down)",2
408,0.305071,"(.DXY_Up, USDNOK_Up)",2
138,0.302377,"(AUDUSD_Up, NZDUSD_Up)",2
480,0.299525,"(USDNOK_Down, .DXY_Down)",2
1278,0.293502,"(JPY_slope_Up, JPY_1Y_Up)",2
1331,0.291918,"(JPY_slope_Down, JPY_1Y_Down)",2
203,0.290808,"(USDNOK_Up, AUDUSD_Down)",2
800,0.287163,"(EUR1MO=_Up, GBP1MO=_Up)",2


# Creating Rules: First set of Association Rules

Support = How often is a condition true as a share of all observations (trading days).

Confidence(A → C) = How often is a rule true. This corresponds to conditional probability that C(onsequent) has moved as described by the rule given A(ntecedent) has moved as described.

Lift = confidence(A → C)/support(C). How much more likely is C to have moved consistent with the rule on a day where A behaved as described.

Leverage = support(A → C)−support(A) * support(C). Range [-1,1]. Difference between observed frequency to base case of A,B being independent.

In [191]:
frequent_itemsets = apriori(DailyChg, min_support=0.1, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules.sort_values(by=['confidence','lift'],ascending=False,inplace=True)
rules[ (rules['antecedent_len'] >= 2)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
44674,"(GBP_slope_Up, EUR_slope_Up, GBP_curve_Up, EUR...",(GBP_1Y_Up),0.106339,0.297623,0.103011,0.968703,3.254802,0.071362,22.442623,4
37554,"(GBP_slope_Up, GBP_curve_Up, EUR_1Y_Up)",(GBP_1Y_Up),0.113788,0.297623,0.109984,0.966574,3.247647,0.076118,21.012784,3
44800,"(GBP_slope_Down, EUR_1Y_Down, GBP_curve_Down, ...",(GBP_1Y_Down),0.103645,0.287956,0.100000,0.964832,3.350627,0.070155,20.246827,4
44758,"(GBP_slope_Down, EUR_1Y_Down, NZD_1Y_Down, EUR...",(GBP_1Y_Down),0.106815,0.287956,0.102694,0.961424,3.338793,0.071936,18.458381,4
38183,"(GBP_1Y_Up, EUR_curve_Up, EUR_slope_Up)",(GBP_slope_Up),0.113312,0.302536,0.108875,0.960839,3.175953,0.074594,17.810250,3
44653,"(GBP_1Y_Up, EUR_slope_Up, EUR_curve_Up, EUR_1Y...",(GBP_slope_Up),0.107765,0.302536,0.103487,0.960294,3.174152,0.070884,17.565769,4
37830,"(GBP_slope_Down, EUR_1Y_Down, NZD_1Y_Down)",(GBP_1Y_Down),0.114897,0.287956,0.110301,0.960000,3.333847,0.077216,17.801109,3
37913,"(GBP_curve_Down, EUR_1Y_Down, GBP_slope_Down)",(GBP_1Y_Down),0.110618,0.287956,0.106181,0.959885,3.333449,0.074328,17.750249,3
44822,"(GBP_1Y_Down, EUR_1Y_Down, NZD_slope_Down, EUR...",(GBP_slope_Down),0.106022,0.292868,0.101743,0.959641,3.276697,0.070693,17.521148,4
38386,"(NOK_slope_Down, GBP_1Y_Down, EUR_slope_Down)",(GBP_slope_Down),0.104596,0.292868,0.100317,0.959091,3.274818,0.069684,17.285438,3


In [193]:
#Interesting rule example: risk off
#SP500 down and JPY 1m ATM vol up -->XAUUSD up
print('antecedents: ' + str(rules.loc[9644].antecedents))
print('consequents: ' + str(rules.loc[9644].consequents))
print('lift: ' + "{0:.2f}".format(round(rules.loc[9644].lift,2)) + ' confidence: ' + "{0:.2f}".format(round(rules.loc[9644].confidence,2)))

antecedents: frozenset({'JPY1MO=_Up', 'S&P 500 (USD)_Down'})
consequents: frozenset({'XAUUSD_Up'})
lift: 1.30 confidence: 0.50


### Rules with significant lift

In [194]:
rules[ (rules['antecedent_len'] >= 3) & (rules['lift'] > 3) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
44674,"(GBP_slope_Up, EUR_slope_Up, GBP_curve_Up, EUR...",(GBP_1Y_Up),0.106339,0.297623,0.103011,0.968703,3.254802,0.071362,22.442623,4
37554,"(GBP_slope_Up, GBP_curve_Up, EUR_1Y_Up)",(GBP_1Y_Up),0.113788,0.297623,0.109984,0.966574,3.247647,0.076118,21.012784,3
44800,"(GBP_slope_Down, EUR_1Y_Down, GBP_curve_Down, ...",(GBP_1Y_Down),0.103645,0.287956,0.100000,0.964832,3.350627,0.070155,20.246827,4
44758,"(GBP_slope_Down, EUR_1Y_Down, NZD_1Y_Down, EUR...",(GBP_1Y_Down),0.106815,0.287956,0.102694,0.961424,3.338793,0.071936,18.458381,4
38183,"(GBP_1Y_Up, EUR_curve_Up, EUR_slope_Up)",(GBP_slope_Up),0.113312,0.302536,0.108875,0.960839,3.175953,0.074594,17.810250,3
44653,"(GBP_1Y_Up, EUR_slope_Up, EUR_curve_Up, EUR_1Y...",(GBP_slope_Up),0.107765,0.302536,0.103487,0.960294,3.174152,0.070884,17.565769,4
37830,"(GBP_slope_Down, EUR_1Y_Down, NZD_1Y_Down)",(GBP_1Y_Down),0.114897,0.287956,0.110301,0.960000,3.333847,0.077216,17.801109,3
37913,"(GBP_curve_Down, EUR_1Y_Down, GBP_slope_Down)",(GBP_1Y_Down),0.110618,0.287956,0.106181,0.959885,3.333449,0.074328,17.750249,3
44822,"(GBP_1Y_Down, EUR_1Y_Down, NZD_slope_Down, EUR...",(GBP_slope_Down),0.106022,0.292868,0.101743,0.959641,3.276697,0.070693,17.521148,4
38386,"(NOK_slope_Down, GBP_1Y_Down, EUR_slope_Down)",(GBP_slope_Down),0.104596,0.292868,0.100317,0.959091,3.274818,0.069684,17.285438,3


In [195]:
#Interesting rule example: risk off
#FXSwap implied yield curves: steeper GBP & EUR ;larger GBP FXS yield curvature & higher 1Y EUR --> higher GBP 1Y yield
print('antecedents: ' + str(rules.loc[44674].antecedents))
print('consequents: ' + str(rules.loc[44674].consequents))
print('lift: ' + "{0:.2f}".format(round(rules.loc[44674].lift,2)) + ' confidence: ' + "{0:.2f}".format(round(rules.loc[44674].confidence,2)))

antecedents: frozenset({'GBP_slope_Up', 'EUR_slope_Up', 'GBP_curve_Up', 'EUR_1Y_Up'})
consequents: frozenset({'GBP_1Y_Up'})
lift: 3.25 confidence: 0.97


In [196]:
#Interesting rule example: risk off
print('antecedents: ' + str(rules.loc[45849].antecedents))
print('consequents: ' + str(rules.loc[45849].consequents))
print('lift: ' + "{0:.2f}".format(round(rules.loc[45849].lift,2)) + ' confidence: ' + "{0:.2f}".format(round(rules.loc[45849].confidence,2)))

antecedents: frozenset({'EURO STOXX 50 VOL_Up', 'MSCI EM_Down', 'S&P 500 (USD)_Down'})
consequents: frozenset({'AUD1MO=_Up', 'VIX_Up', 'EURO STOXX 50 (EUR)_Down'})
lift: 3.18 confidence: 0.52
